# Set up the environment

In [ ]:
import numpy as np
import os
import json
import matplotlib.pyplot as plt
import seaborn
from sklearn.manifold import TSNE, MDS

# Define a function to create a filename


In [ ]:
def getFileName(name, n_samples, model_name, layer_name):
    return name \
        + "_{}_".format(n_samples) \
        + "_{}_".format(model_name) \
        + "_{}".format(layer_name)  \
        + ".npy"   

# Get the model and layers' names and load the model RDM

In [ ]:
#load the np file containing the shape of the activations
ROOT_PATH = '/mnt/raid/ni/agnessa/RSA/'
NR_OF_SAMPLES = 10000
json_file_layers=os.path.join(ROOT_PATH,'resnets_selected_layers.json')
with open(json_file_layers, "r") as fd:
    selected_layers = json.load(fd)

#get the name of the model(s) and of the layers
num_layers = 16
model_begin_1 = 0 #index of the first layer of the desired model
model_name_1 = selected_layers[model_begin_1].get('model')
model_begin_2 = 16
model_name_2 = selected_layers[model_begin_2].get('model')
layer_names_1 = []
layer_names_2 = []

#append the layers from the first model
for i in range(num_layers):
    layer_names_1.append(selected_layers[model_begin_1+i].get('layer'))   
    
#append the layers from the second model
for i in range(num_layers):
    layer_names_2.append(selected_layers[model_begin_2+i].get('layer'))
    
#Load the model RDM
layer_name_model_rdm = 'all'
model_name = model_name_1 + '_' + model_name_2
path = os.path.join(ROOT_PATH + 'Model_RDM/', getFileName("Model_RDM",NR_OF_SAMPLES,model_name,layer_name_model_rdm))
Model_RDM = np.load(path)

# Plot the Model RDM


In [ ]:
print("Plotting Model RDM for model -> ", model_name)
Model_RDM[np.isnan(Model_RDM)]=0.0 
fig = plt.figure(figsize=(17,15))
ax = seaborn.heatmap(Model_RDM, xticklabels = layer_names_2, yticklabels = layer_names_1, cmap='rainbow', vmin=0.0, vmax=1.0)
plt.xticks(rotation=90) 
plt.yticks(rotation=0)
plt.show()

# Save as a .png file

In [ ]:
path = os.path.join(ROOT_PATH + 'Model_RDM_plots', getFileName("Model_RDM_1.0",NR_OF_SAMPLES,model_name,layer_name_model_rdm) + '.png')
fig.savefig(path)

# Multidimensional scaling

In [ ]:
mds = MDS(n_components=2, n_init=200, max_iter=2000, eps=0.0001, dissimilarity='precomputed')
data_embedded = mds.fit_transform(Model_RDM)

In [ ]:
sorted_layers = [1.0,1.1,1.2,2.0,2.1,2.2,2.3,3.0,3.1,3.2,3.3,3.4,3.5,4.0,4.1,4.2]
fig, ax = plt.subplots()
sc = ax.scatter(data_embedded[:, 0], data_embedded[:, 1], c=sorted_layers, cmap = 'rainbow')

for i in range(np.array(layer_names).shape[0]):
    ax.annotate(layer_names[i], (data_embedded[i, 0], data_embedded[i, 1]))

plt.show()

In [ ]:
path = os.path.join(ROOT_PATH + 'Model_RDM_MDS', getFileName("Model_RDM_MDS",NR_OF_SAMPLES,model_name,layer_name_model_rdm) + '.png')
fig.savefig(path)